In [1]:
import sys
sys.path.append('/home/luis-felipe/brats_lf/src')
sys.path.insert(1, '../src')

In [2]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold

import utils

In [3]:
PROJ_ROOT = os.path.join(os.path.abspath(os.pardir))
PROJ_ROOT

'/home/luis-felipe/brats_lf'

In [4]:
class GlobalConfig:
    
    root_dir = os.path.join(PROJ_ROOT,'data')
    train_root_dir =os.path.join(root_dir, 'BraTS2020_TrainingData','MICCAI_BraTS2020_TrainingData')
    test_root_dir = os.path.join(root_dir, 'BraTS2020_TrainingData','MICCAI_BraTS2020_ValidationData')
    path_to_csv = os.path.join(root_dir,'train_data.csv')
    pretrained_model_path = os.path.join(PROJ_ROOT,'brats2020logs','unet','last_epoch_model.pth')
    train_logs_path = os.path.join(PROJ_ROOT,'brats2020logs','unet','train_log.csv')
    ae_pretrained_model_path = os.path.join(PROJ_ROOT,'brats2020logs','ae','autoencoder_best_model.pth')
    tab_data = os.path.join(PROJ_ROOT,'brats2020logs','data','df_with_voxel_stats_and_latent_features.csv')
    seed = 55

    
config = GlobalConfig()
utils.seed_everything(config.seed)

In [5]:
root_dir = os.path.join(PROJ_ROOT,'data')
train_root_dir =os.path.join(root_dir, 'BraTS2020_TrainingData','MICCAI_BraTS2020_TrainingData')
    
survival_info_df = pd.read_csv(os.path.join(train_root_dir,'survival_info.csv'))
name_mapping_df =pd.read_csv(os.path.join(train_root_dir,'name_mapping.csv'))

name_mapping_df.rename({'BraTS_2020_subject_ID': 'Brats20ID'}, axis=1, inplace=True) 


df = survival_info_df.merge(name_mapping_df, on="Brats20ID", how="right")

paths = []
for _, row  in df.iterrows():
    
    id_ = row['Brats20ID']
    phase = id_.split("_")[-2]
    
    if phase == 'Training':
        path = os.path.join(config.train_root_dir, id_)
    else:
        path = os.path.join(config.test_root_dir, id_)
    paths.append(path)
    
df['path'] = paths

#split data on train, test, split
#train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=69, shuffle=True)
#train_df, val_df = train_df.reset_index(drop=True), val_df.reset_index(drop=True)

train_data = df.loc[df['Age'].notnull()].reset_index(drop=True)
train_data["Age_rank"] =  train_data["Age"] // 10 * 10
train_data = train_data.loc[train_data['Brats20ID'] != 'BraTS20_Training_355'].reset_index(drop=True, )

skf = StratifiedKFold(
    n_splits=7, random_state=config.seed, shuffle=True
)
for i, (train_index, val_index) in enumerate(
        skf.split(train_data, train_data["Age_rank"])
        ):
        train_data.loc[val_index, "fold"] = i

train_df = train_data.loc[train_data['fold'] != 0].reset_index(drop=True)
val_df = train_data.loc[train_data['fold'] == 0].reset_index(drop=True)

test_df = df.loc[~df['Age'].notnull()].reset_index(drop=True)
print("train_df ->", train_df.shape, "val_df ->", val_df.shape, "test_df ->", test_df.shape)
train_data.to_csv(config.path_to_csv, index=False)

train_df -> (201, 12) val_df -> (34, 12) test_df -> (133, 10)


/home/luis-felipe/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(("The least populated class in y has only %d"
